In [1]:
import pandas as pd
import numpy as np
import pickle,re,random
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

df=pd.read_csv('url.csv')

#DATA CLEANING
df=df.dropna()
df=df.drop_duplicates()

train,test=train_test_split(df,train_size=0.8,random_state=40)
train=train.reset_index(drop=True)
test=test.reset_index(drop=True)

In [2]:
# --Removing un wanted words to train 
def rtokenize(url):  
    tokens = re.split('[/-]', url)
    if(tokens[0]=="https:" or tokens[0]=="http:"):
        tokens.pop(0)
        
    if(tokens[0]==""):
        tokens.pop(0)
    
    for i in tokens:
        if i.find(".") >= 0:
            tokens.remove(i)
            dot_split = i.split('.')
            if "com" in dot_split:
                dot_split.remove("com")
            if "www" in dot_split:
                dot_split.remove("www")
            tokens += dot_split  
    return ' '.join(tokens)

train['URLs']=train['URLs'].apply(rtokenize)

In [3]:
# Count Vectorizer on data
cv=CountVectorizer()
cv.fit(train['URLs'])
a=cv.transform(train['URLs'])

#dump count Vectorizer
out=open("vectorizer.pickle","wb")
pickle.dump(cv,out)
out.close()

In [4]:
#Classifier with count vectorizer
mb_cv = MultinomialNB()
mb_cv.fit(a, train['Class'])

#dump Classifier
out=open("phishing.pickle","wb")
pickle.dump(mb_cv,out)
out.close()

In [5]:
#Testing with a new input
import pickle
input="genesysproperties.co.ke/john/cp.php?m=login"
pred=[rtokenize(input)]    
vecf=open("vectorizer.pickle","rb")
vec=pickle.load(vecf)
vec=vec.transform(pred)

clff=open("phishing.pickle","rb")
clf=pickle.load(clff)
pred_mb_cv=clf.predict(vec)
print(pred_mb_cv[0])

bad
